In [ ]:
from ast import literal_eval
from collections import defaultdict
import pandas as pd

# 데이터 불러오기
tagged_df = pd.read_csv("tagged_review_data_전체신발.csv")

# 감성 카테고리 정의
category_lists = {
   '편한' : [
	'편하다', '편안하다'
    ],
   '푹신한' : [
	'푹신하다'
    ],
   '가벼운' : [
	'가볍다'
    ],
   '부드러운' : [
	'부드럽다'
    ],
   '플랫한' : [
	'딱딱하다'
    ],
   '예쁜' : [
	'이쁘다', '예쁘다'
    ],
   '귀여운' : [
	'귀엽다'
    ],
   '깔끔한' : [
	'깔끔하다'
    ],
   '모던한' : [
	'세련되다', '멋지다', '고급스럽다'
    ],
   '휘뚜루마뚜루': [
        '데일', '자주', '매일', '코디', '활용', '어울리다', '여기저기', '포인트', '무난'
    ],
    '발볼이 넓은' : [
	'크다', '다운', '넉넉하다', '넓다'
    ],
    '발볼이 보통' : [
	'정사', '잘맞다', '딱이다', '맞다'
    ],
    '발볼이 좁은' : [
	'업', '작다', '좁다'
    ],
    '가격이 착한': [
        '가성', '저렴하다', '싸다', '할인', '쿠폰', '득템', '대비', '착한', '착하다'
    ],
    '튼튼한': [
        '튼튼하다', '구성', '구도', '마감', '퀄리티', '재질', '오래오래'
    ]
}

# word → category 매핑
word_to_category = {}
for category, keywords in category_lists.items():
    for word in keywords:
        word_to_category[word] = category

# 1. 카테고리 빈도 누적: {product_id: {category: count}}
product_scores = defaultdict(lambda: defaultdict(int))

for _, row in tagged_df.iterrows():
    product_id = row['id']
    try:
        tokens = literal_eval(row['tagged_review'])
    except:
        continue
    for token in tokens:
        category = word_to_category.get(token)
        if category:
            product_scores[product_id][category] += 1

# 2. 제품별 리뷰 수 계산
review_counts = tagged_df['id'].value_counts().to_dict()

# 3. 결과 정리: 빈도 + 백분율
results = []
for product_id, cat_counts in product_scores.items():
    row = {'product_id': product_id}
    for category in category_lists.keys():
        freq = cat_counts.get(category, 0)
        percent = (freq / review_counts.get(product_id, 1)) * 100
        row[f"{category}_빈도"] = freq
        row[f"{category}_비율(%)"] = round(percent, 2)
    results.append(row)

# 4. 데이터프레임 변환 및 저장
final_df = pd.DataFrame(results)

# 빈도 컬럼만 추출
freq_cols = [col for col in final_df.columns if '빈도' in col and '비율' not in col]
freq_df = final_df[['product_id'] + freq_cols]

# 비율 컬럼만 추출
percent_cols = [col for col in final_df.columns if '비율' in col]
percent_df = final_df[['product_id'] + percent_cols]

# 저장
freq_df.to_csv("제품별_카테고리_빈도.csv", index=False)
percent_df.to_csv("제품별_카테고리_비율.csv", index=False)

